In [2]:
import tkinter as tk
import pandas as pd
from tkinter import filedialog

root = tk.Tk()
file=filedialog.askopenfilename()
root.withdraw()

df=pd.read_excel(file, skiprows=5, skipfooter=2, header=0)

In [3]:
#Deleting 3rd row of each participant
df.dropna(subset=['کد ملی'], inplace=True)
df.reset_index(drop=True, inplace=True)

#Creating a df from even rows
df1 = df[df.index%2 == 0]
df1.iloc[0,[8,9,11]]=["Birth_En","Mobile","Name_En"]
df1.columns=df1.iloc[0]
df1=df1.drop(df1.index[0])
df1.reset_index(drop=True, inplace=True)

#Creating a df from odd rows
df2 = df[df.index%2 == 1]
df2.reset_index(drop=True, inplace=True)
df2.columns.values[[6,8,9,11]]=["Prov","Birth_Fa","Nat_ID","Name_Fa"]

#Prearing a neat df
df_concat = pd.concat([df1, df2], axis=1)
df_concat=df_concat.iloc[:,[26,11,21,23,8,9,24]]

/home/mohammad/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [4]:
#Dataframe to Dictionary
df_dict=df_concat.to_dict(orient='series')

In [5]:
#Check and report the errors
Errors=""

for value in df_dict['Name_Fa'].items():
    try:
        not(str(value[1]).isdigit() and str(value[1]).isascii())
    except:
        Errors+= str(value[1]) + " in row number " + str(value[0]+1) + " is not Persian." + "\n"
    
for value in df_dict['Name_En'].items():
    try:
        str(value[1]).title().isascii()
    except:
        Errors+= str(value[1]) + " in row number " + str(value[0]+1) + " is not English." + "\n"
        
for value in df_dict['Prov'].items():
    try:
        not(str(value[1]).isdigit() and str(value[1]).isascii())
    except:
        Errors+= str(value[1]) + " in row number " + str(value[0]+1) + " is not Persian." + "\n"

for value in df_dict['Birth_Fa'].items():
    try:
        str(value[1]).replace(" ","").isdigit() 
    except:
        Errors+= str(value[1]) + " in row number " + str(value[0]+1) + " is not a valid date like 1399/01/01." + "\n"

for value in df_dict['Birth_En'].items():
    try:
        str(value[1]).replace(" ","").isdigit()
    except:
        Errors+= str(value[1]) + " in row number " + str(value[0]+1) + " is not a valid date." + "\n"
        
for value in df_dict['Mobile'].items():
    try:
        str(value[1]).isdigit() and len(str(value[1]))==11
    except:
        Errors+= str(value[1]) + " in row number " + str(value[0]+1) + " is not a valid Mobile Phone." + "\n"
        
for value in df_dict['Nat_ID'].items():
    try:
        str(value[1]).isdigit() and len(str(value[1]))==10
    except:
        Errors+= str(value[1]) + " in row number " + str(value[0]+1) + " is not a valid National ID." + "\n"

In [12]:
#Dictionary to class
class Besmela:
    #def __init__(self, Dict):
        for k, v in Dict.items():
            setattr(self, k, v)

df_class=Besmela(df_dict)
#print(df_dict.items())
#dir(df_class)

print(type(df_class.Birth_Fa))

<class 'pandas.core.series.Series'>


In [ ]:
import sqlite3 as sq
conn= sq.connect('competition.db')
cur= conn.cursor()
cur.execute(""" CREATE TABLE athletes_info5 (Name_Fa, Name_En, Prov, Birth_Fa, Birth_En, Mobile, Nat_ID)""")
cur.execute("""INSERT INTO athletes_info VALUES (?,?,?,?,?,?,?)""", df_class)
conn.commit()